# Test on Guru Dataset
How robust are all previously developped models, regarding the selfmade guru dataset?

**Conclusion:** Performance recieves a shock.

In [1]:
# Compute on CPU (GPU is not available)
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import keras
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import kapre
from utils import DataFeed
import sys
sys.path.append('../preprocessing')

Using TensorFlow backend.


 ## Create Dataset from raw data

In [3]:
from helper_functions import *
import os

In [4]:
raw_data_path ='../data/raw/guru'
data_path = '../preprocessing/preprocessed_data'

In [5]:
for lang in ['de', 'fr', 'en']:
    for path in get_paths(os.path.join(raw_data_path, lang)):
        part, sr = librosa.load(path, sr=16000, duration=5.0)
        np.save(os.path.join(data_path, 'test/guru', lang, os.path.basename(path)), part)

## Load Data

In [5]:
data, labels = DataFeed.Dataset.create(data_path, ['test/guru'], num=-1, shuffle=True)
data.shape

(70, 1, 80000)

In [6]:
import IPython.display as ipd
ipd.Audio(data[-2], rate=16000)

## Evaluate

In [7]:
models = ['conv2d_to_rnn/conv2d_to_rnn.h5', 'berlin_net/berlin_net_big.h5', 'conv2d_deep/conv2d_deep.h5']

In [8]:
for mpath in models:
    model = load_model(mpath, custom_objects={'Melspectrogram':kapre.time_frequency.Melspectrogram, 
                                              'Normalization2D': kapre.utils.Normalization2D})
    results = model.evaluate(x=data, y=labels, verbose=0)
    print(f"{mpath}: {model.metrics_names[0]} = {results[0]},  {model.metrics_names[1]} = {results[1]}")

int_axis=0 passed but is ignored, str_axis is used instead.
conv2d_to_rnn/conv2d_to_rnn.h5: loss = 0.6187191426753997,  acc = 0.8571428571428571
int_axis=0 passed but is ignored, str_axis is used instead.
berlin_net/berlin_net_big.h5: loss = 0.6740766808390617,  acc = 0.8285714285714286


/home/joel/.local/share/virtualenvs/deepLID-B2i2hIZa/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


conv2d_deep/conv2d_deep.h5: loss = 1.1015194462878364,  acc = 0.7142857125827244
